Quiz1


In [ ]:
import torch

In [ ]:
def timer(func): # timer using decorator
  def wrapper(*args, **kwargs):
    start_time = time.time()
    result = func(*args, **kwargs)
    end_time = time.time()
    computation_time = end_time - start_time
    print(f"Execution time of {func.__name__}: {computation_time} seconds")
    return result
  return wrapper

In [ ]:
import numpy as np
import time

In [ ]:
true_b = 1
true_w = 2
N = 100

np.random.seed(42)

#N개 난수 생성, 정규분포 따름
x = np.random.rand(N,1)
epsilon = 0.1*np.random.rand(N,1)
y = true_b + true_w*x + epsilon


# trainset, validationset
idx = np.arange(N)
split_index = int(N*0.8)
train_index = idx[:split_index]
val_index = idx[split_index:]

x_train, y_train = x[train_index], y[train_index]

x_val, y_val = x[val_index], y[val_index]


#create tensor at CPU
x_train_tensor = torch.as_tensor(x_train)
y_train_tensor = torch.as_tensor(y_train)

#create tensor at GPU
device = "cuda" if torch.cuda.is_available() else "cpu"
##GPU로 보냄
x_train_tensor = torch.as_tensor(x_train).to(device)
y_train_tensor = torch.as_tensor(y_train).to(device)


from torch import nn, optim

@timer
def train_model_optim(lr = 0.1, epochs = 1000):
    b = torch.randn(1, requires_grad=True, dtype=torch.float, device=device)
    w = torch.randn(1, requires_grad=True, dtype=torch.float, device=device)

    parameters = [b,w]
    optimizer = optim.SGD(parameters, lr=lr)
    mse_loss = nn.MSELoss()

    for epoch in range(epochs):
        #loss
        y_hat = b + w * x_train_tensor
        loss = mse_loss(y_hat, y_train_tensor)


        #gradient
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
    return w, b



train_w, train_b = train_model_optim()
print(train_w, train_b)



###1
x_val_tensor = torch.as_tensor(x_val).to(device)
y_val_tensor = torch.as_tensor(y_val).to(device)


mse_loss = nn.MSELoss()
y_hat = train_b + train_w * x_val_tensor
loss = (mse_loss(y_hat, y_val_tensor))
print(loss)


Quiz2


In [ ]:
import time

def timer(func): # timer using decorator
  def wrapper(*args, **kwargs):
    start_time = time.time()
    result = func(*args, **kwargs)
    end_time = time.time()
    computation_time = end_time - start_time
    print(f"Execution time of {func.__name__}: {computation_time} seconds")
    return result
  return wrapper

In [ ]:
import pickle

with open('./sample_data/quiz_data.pkl', "rb") as fr:
    data = pickle.load(fr)



In [ ]:
import numpy as np
import torch
from torch import nn, optim

N = 100

idx = np.arange(N)

split_index = int(N*0.8)
train_index = idx[:split_index]
val_index = idx[split_index:]

x_train, y_train = data['x'][train_index], data['y'][train_index]
x_val, y_val = data['x'][val_index], data['y'][val_index]



In [ ]:
x_train_tensor = torch.as_tensor(x_train)
y_train_tensor = torch.as_tensor(y_train)
x_val_tensor = torch.as_tensor(x_val)
y_val_tensor = torch.as_tensor(y_val)

x_mean = x_train_tensor.mean()
x_std = x_train_tensor.std()
x_train_tensor = (x_train_tensor - x_mean) / x_std

y_mean = y_train_tensor.mean()
y_std = y_train_tensor.std()
y_train_tensor = (y_train_tensor - y_mean) / y_std

x_mean = x_val_tensor.mean()
x_std = x_val_tensor.std()
x_val_tensor = (x_val_tensor - x_mean) / y_std

y_mean = y_val_tensor.mean()
y_std = y_val_tensor.std()
y_val_tensor = (y_val_tensor - y_mean) / y_std

#create tensor at GPU
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)
##GPU로 보냄
x_train_tensor = torch.as_tensor(x_train).to(device)
y_train_tensor = torch.as_tensor(y_train).to(device)
x_val_tensor = torch.as_tensor(x_val).to(device)
y_val_tensor = torch.as_tensor(y_val).to(device)


In [ ]:
@timer
def train_model_optim(lr = 0.5, epochs = 1000):
    w1 = torch.randn(1, requires_grad=True, dtype=torch.float, device=device)
    w2 = torch.randn(1, requires_grad=True, dtype=torch.float, device=device)
    w3 = torch.randn(1, requires_grad=True, dtype=torch.float, device=device)
    w4 = torch.randn(1, requires_grad=True, dtype=torch.float, device=device)
    w5 = torch.randn(1, requires_grad=True, dtype=torch.float, device=device)
    b = torch.randn(1, requires_grad=True, dtype=torch.float, device=device)

    parameters = [b,w1, w2, w3, w4, w5]
    optimizer = optim.SGD(parameters, lr=lr)
    mse_loss = nn.MSELoss()

    for epoch in range(epochs):
        #loss
        y_hat = b + w1 * x_train_tensor + w2 * (x_train_tensor ** 2) + w3 * (x_train_tensor ** 3) + w4 * (x_train_tensor ** 4) + w5 * (x_train_tensor ** 5)
        loss = mse_loss(y_hat, y_train_tensor)
        # print("loss: ", loss)

        #gradient
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
    return w1, w2, w3, w4,w5, b

In [ ]:
train_w1, train_w2,train_w3,train_w4, train_w5,train_b = train_model_optim()
print(train_w1, train_w2,train_w3,train_w4,train_w5, train_b)

In [ ]:
mse_loss = nn.MSELoss()
num = 10
loss_sum = 0
lr = 0.5
epochs = 2000

for i in range(num):
  train_w1, train_w2, train_w3, train_w4, train_w5, train_b = train_model_optim(lr=lr, epochs=epochs)

  with torch.no_grad():
    y_hat = train_b + train_w1 * x_val_tensor + train_w2 * (x_val_tensor ** 2) + train_w3 * (x_val_tensor ** 3) + train_w4 * (x_val_tensor ** 4) + train_w5 * (x_val_tensor ** 5)
    loss = mse_loss(y_hat, y_val_tensor)
    loss_sum += loss.item()  # MSE 값을 누적
    print("loss: ", loss)

average = loss_sum / num
print("Average loss: ", average)